## 154 Term

In [4]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error
from mv_wg_avg import exp_weight_avg


os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Data Load

In [5]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
train_test = pd.concat([df_train, df_test], axis=0)
df_hist = utils.read_df_pkl('../input/hist*0*')
df_new = utils.read_df_pkl('../input/new*0*')

df_train = reduce_mem_usage(df_train)
df_test  = reduce_mem_usage(df_test )
train_test  = reduce_mem_usage(train_test )
df_hist  = reduce_mem_usage(df_hist )
df_new   = reduce_mem_usage(df_new  )

100%|██████████| 3/3 [00:01<00:00,  2.36it/s]


Mem. usage decreased to  4.04 Mb (56.2% reduction)
Mem. usage decreased to  2.24 Mb (52.5% reduction)
Mem. usage decreased to  6.52 Mb (56.2% reduction)
Mem. usage decreased to 1971.22 Mb (40.8% reduction)
Mem. usage decreased to 140.41 Mb (37.5% reduction)


In [19]:
# tmp = df_hist['purchase_amount'].values/100000
# tmp = tmp[tmp<0.001]
# tmp = pd.Series(np.round(tmp, 4)).value_counts()
tmp =df_hist['purchase_amount']
tmp[tmp>0].shape

(1075467,)

In [10]:
hist_max = df_hist[[key, 'purchase_date']].groupby(key).max()
new_max = df_new[[key, 'purchase_date']].groupby(key).max()
date_max = pd.concat([hist_max, new_max], axis=0).reset_index()
date_max = date_max.groupby(key)['purchase_date'].max()
date_max.name = 'last_active_date'

In [11]:
df_hist = df_hist.merge(train_test, how='inner', on=key).merge(date_max.reset_index(), how='inner', on=key)
df_new = df_new.merge(train_test, how='inner', on=key).merge(date_max.reset_index(), how='inner', on=key)
for df in [df_hist, df_new]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['authorized_flag'] = df['authorized_flag'].map(lambda x: 0 if x=='N' else 1)
    df['all_term'] = (pd.to_datetime('2018-05-01') - df['first_active_month']).dt.days
    df['personal_term'] = (pd.to_datetime(df['last_active_date']) - df['first_active_month']).dt.days
#     df['all_term'] = (datetime.datetime.today() - df['first_active_month']).dt.days
df_hist.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,feature_1,feature_2,feature_3,first_active_month,target,all_term,last_active_date,personal_term
0,1,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37,3,3,1,2016-06-01,NaN,699.0,2018-04-29 18:13:17,697.0
1,1,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16,3,3,1,2016-06-01,NaN,699.0,2018-04-29 18:13:17,697.0
2,1,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37,3,3,1,2016-06-01,NaN,699.0,2018-04-29 18:13:17,697.0
3,1,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34,3,3,1,2016-06-01,NaN,699.0,2018-04-29 18:13:17,697.0
4,1,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37,3,3,1,2016-06-01,NaN,699.0,2018-04-29 18:13:17,697.0


In [14]:
df_hist['freq'] = 1
df_new['freq'] = 1
auth1 = df_hist[df_hist['authorized_flag']==1]
auth0 = df_hist[df_hist['authorized_flag']==0]
df_trans = pd.concat([auth1, df_new], axis=0)
df_trans = reduce_mem_usage(df_trans)

Mem. usage decreased to 2560.14 Mb (24.2% reduction)


In [16]:
df_term = df_trans[[key, 'all_term', 'personal_term']].drop_duplicates().set_index(key)

for fname, df in zip(['auth1', 'auth0', 'new', 'all'], [auth1, auth0, df_new, df_trans]):
    tmp = df.groupby(key)['freq'].sum()
    tmp2 = pd.concat([tmp, df_term], axis=1)
#     tmp2[f'{fname}_freq_per_days_all_term'] = tmp2['freq'] /  tmp2['all_term']
    tmp2[f'{fname}_freq_per_days_personal_term'] = tmp2['freq'] /  tmp2['personal_term']
    
#     df_train = df_train.join(tmp2[[f'{fname}_freq_per_days_all_term']])
#     df_test = df_test.join(tmp2[[f'{fname}_freq_per_days_all_term']])
    df_train = df_train.join(tmp2[[f'{fname}_freq_per_days_personal_term']])
    df_test = df_test.join(tmp2[[f'{fname}_freq_per_days_personal_term']])
    
df_train = df_train.join(df_term)
df_test = df_test.join(df_term)
df_train.head()

,first_active_month,feature_1,feature_2,feature_3,target,auth1_freq_per_days_personal_term,auth0_freq_per_days_personal_term,new_freq_per_days_personal_term,all_freq_per_days_personal_term,all_term,personal_term
card_id,,,,,,,,,,,
C_ID_92a2005557,2017-06,5,2,1,-0.820312,0.743976,0.039157,0.069277,0.813253,334.0,332.0
C_ID_3d0044924f,2017-01,4,1,0,0.392822,0.748344,0.024283,0.013245,0.761589,485.0,453.0
C_ID_d639edf6cd,2016-08,2,2,0,0.687988,0.064567,0.003150,0.001575,0.066142,638.0,635.0
C_ID_186d6a6901,2017-09,4,3,0,0.142456,0.336245,NaN,0.030568,0.366812,242.0,229.0
C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159790,0.719101,0.028090,0.202247,0.921348,181.0,178.0


### Save feature

In [17]:
fname = '154_trm'
fname = '157_trm'
ignore_features = ['first_active_month', 'card_id', target]

for col in df_train.columns:
    if not(col.count('personal')):continue
#     length = len(df_train[col].dropna())
#     if length/len(df_train)<0.1:
#         print(col)
#         continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_test_{col}@', obj=df_test[col].values)

In [23]:
import glob
pl = glob.glob('../features/1_first_valid/*.gz')

for p in pl:
    print(p)
    print(utils.read_pkl_gzip(p).shape)

../features/1_first_valid/155_trm_test_new_freq_per_days_all_term@.gz
(123623,)
../features/1_first_valid/153_nlc_test_new_merchant_category_id_null_cnt@.gz
(123623,)
../features/1_first_valid/154_trm_test_auth1_freq_per_days_all_term@.gz
(123623,)
../features/1_first_valid/129_tzr_test_timezone_ratio_night@.gz
(123623,)
../features/1_first_valid/129_tzr_test_timezone_ratio_afternoon@.gz
(123623,)
../features/1_first_valid/154_trm_train_new_freq_per_days_all_term@.gz
(201917,)
../features/1_first_valid/153_nlc_train_auth1_city_id_null_cnt@.gz
(201917,)
../features/1_first_valid/153_nlc_train_new_installments_null_cnt@.gz
(201917,)
../features/1_first_valid/155_trm_train_new_freq_per_days_all_term@.gz
(201917,)
../features/1_first_valid/155_trm_train_auth1_freq_per_days_all_term@.gz
(201917,)
../features/1_first_valid/154_trm_test_auth0_freq_per_days_all_term@.gz
(123623,)
../features/1_first_valid/155_trm_test_all_term@.gz
(123623,)
../features/1_first_valid/155_trm_train_all_freq_per_